# NETWORK INTRUSION DETECTION

In [1]:
import pandas as pd
import numpy as np

In [2]:
data_tr=pd.read_csv('G:/AnalytixLab/PYTHON & Machine learning/Assignment/M.L assignments/Supervised Learning (Regression, Classification, Forecasting)/3. Network Intrusion Detection System/NSL_Dataset/Train.txt',header=None)
data_ts=pd.read_csv('G:/AnalytixLab/PYTHON & Machine learning/Assignment/M.L assignments/Supervised Learning (Regression, Classification, Forecasting)/3. Network Intrusion Detection System/NSL_Dataset/Test.txt',header=None)

In [3]:
data_tr.columns=['duration','protocol_type','service','flag','src_bytes','dst_bytes','land', 'wrong_fragment','urgent','hot',
               'num_failed_logins','logged_in', 'num_compromised','root_shell','su_attempted','num_root','num_file_creations', 
               'num_shells','num_access_files','num_outbound_cmds','is_host_login', 'is_guest_login','count','srv_count',
               'serror_rate', 'srv_serror_rate', 'rerror_rate','srv_rerror_rate','same_srv_rate', 'diff_srv_rate',
               'srv_diff_host_rate','dst_host_count','dst_host_srv_count','dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
               'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate','dst_host_serror_rate','dst_host_srv_serror_rate',
               'dst_host_rerror_rate','dst_host_srv_rerror_rate','attack', 'last_flag']

In [4]:
data_tr.tail(3)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack,last_flag
125970,0,tcp,smtp,SF,2231,384,0,0,0,0,...,0.12,0.06,0.0,0.0,0.72,0.0,0.01,0.0,normal,18
125971,0,tcp,klogin,S0,0,0,0,0,0,0,...,0.03,0.05,0.0,0.0,1.00,1.0,0.00,0.0,neptune,20
125972,0,tcp,ftp_data,SF,151,0,0,0,0,0,...,0.30,0.03,0.3,0.0,0.00,0.0,0.00,0.0,normal,21


In [5]:
data_tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 43 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     125973 non-null  int64  
 1   protocol_type                125973 non-null  object 
 2   service                      125973 non-null  object 
 3   flag                         125973 non-null  object 
 4   src_bytes                    125973 non-null  int64  
 5   dst_bytes                    125973 non-null  int64  
 6   land                         125973 non-null  int64  
 7   wrong_fragment               125973 non-null  int64  
 8   urgent                       125973 non-null  int64  
 9   hot                          125973 non-null  int64  
 10  num_failed_logins            125973 non-null  int64  
 11  logged_in                    125973 non-null  int64  
 12  num_compromised              125973 non-null  int64  
 13 

In [5]:
data_tr['attack'].replace(['back','land','neptune','pod','smurf','teardrop','apache2','udpstrom','processtable',
                           'worm(10)'],'DoS',inplace=True)
data_tr['attack'].replace(['satan','ipsweep','nmap','portsweep','mscan','saint(6)'],'Probe',inplace=True)
data_tr['attack'].replace(['guess_passwd','ftp_write','imap','phf','multihop','warezmaster','warezclient','spy','xlock',
                           'xsnoop','snmpguess','snmpgetattack','httptunnel','sendmail','named(16)'],'R2L',inplace=True)
data_tr['attack'].replace(['buffer_overflow','loadmodule','rootkit','perl','sqlattack','xterm','ps(7)'],'U2R',inplace=True)

In [7]:
#data_tr.to_excel(r'C:/Users/Himanshu Shekhar Pal/Desktop/network_data.xlsx',index=False)

In [8]:
data_tr['land']=data_tr['land'].astype('object')
data_tr['logged_in']=data_tr['logged_in'].astype('object')
data_tr['root_shell']=data_tr['root_shell'].astype('object')
data_tr['su_attempted']=data_tr['su_attempted'].astype('object')
data_tr['is_host_login']=data_tr['is_host_login'].astype('object')
data_tr['is_guest_login']=data_tr['is_guest_login'].astype('object')

1. Binomial Classification: Activity is normal or attack

In [9]:
trn_data=data_tr.copy()

In [10]:
trn_data['attack'].replace(['DoS','Probe','R2L','U2R'],'1',inplace=True)
trn_data['attack'].replace(['normal'],'0',inplace=True)

In [11]:
cont_vars=trn_data[['duration','src_bytes','dst_bytes','wrong_fragment','urgent','hot','num_failed_logins','num_compromised',
                   'num_root','num_file_creations','num_shells','num_access_files','num_outbound_cmds','count','srv_count',
                   'serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate',
                   'srv_diff_host_rate','dst_host_count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate',
                   'dst_host_same_src_port_rate','dst_host_srv_diff_host_rate','dst_host_serror_rate',
                   'dst_host_srv_serror_rate','dst_host_rerror_rate','dst_host_srv_rerror_rate','last_flag']]
categ_vars=trn_data[['protocol_type','service','flag','attack','land','logged_in','root_shell','su_attempted','is_host_login',
                     'is_guest_login',]]

In [12]:
def continous_var_summary(x):
    return pd.Series([x.count(),x.isnull().sum(),x.sum(),x.mean(),x.median(),x.std(),x.var(),x.min(),
                      x.dropna().quantile(0.01),x.dropna().quantile(0.05),x.dropna().quantile(0.10),x.dropna().quantile(0.25),
                      x.dropna().quantile(0.50),x.dropna().quantile(0.75),x.dropna().quantile(0.90),x.dropna().quantile(0.95),
                      x.dropna().quantile(0.99),x.max()],index=['N','NMISS','SUM','MEAN','MEDIAN','St~DEV','VAR','MIN','P1',
                                                                'P5','P10','P25','P50','P75','P90','P95','P99','MAX'])
cont_vars.apply(lambda a:continous_var_summary(a)).T.round(2)

,N,NMISS,SUM,MEAN,MEDIAN,St~DEV,VAR,MIN,P1,P5,P10,P25,P50,P75,P90,P95,P99,MAX
duration,125973.0,0.0,3.617247e+07,287.14,0.00,2604.52,6.783500e+06,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,4.00,9590.56,4.290800e+04
src_bytes,125973.0,0.0,5.740179e+09,45566.74,44.00,5870331.18,3.446079e+13,0.0,0.0,0.00,0.00,0.00,44.00,276.00,848.00,1480.00,54540.00,1.379964e+09
dst_bytes,125973.0,0.0,2.491634e+09,19779.11,0.00,4021269.15,1.617061e+13,0.0,0.0,0.00,0.00,0.00,0.00,516.00,3375.80,8314.00,25519.00,1.309937e+09
wrong_fragment,125973.0,0.0,2.858000e+03,0.02,0.00,0.25,6.000000e-02,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.000000e+00
urgent,125973.0,0.0,1.400000e+01,0.00,0.00,0.01,0.000000e+00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.000000e+00
hot,125973.0,0.0,2.575000e+04,0.20,0.00,2.15,4.620000e+00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.00,7.700000e+01
num_failed_logins,125973.0,0.0,1.540000e+02,0.00,0.00,0.05,0.000000e+00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5.000000e+00
num_compromised,125973.0,0.0,3.517800e+04,0.28,0.00,23.94,5.732200e+02,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,7.479000e+03
num_root,125973.0,0.0,3.806800e+04,0.30,0.00,24.40,5.953400e+02,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,7.468000e+03
num_file_creations,125973.0,0.0,1.596000e+03,0.01,0.00,0.48,2.300000e-01,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.300000e+01


In [13]:
# Outlier-handling
cont_vars2=cont_vars.apply(lambda x: x.clip(lower=x.quantile(0.01),upper=x.quantile(0.99)))

In [ ]:
def create_dummies(df,colname):
    col_dummies=pd.get_dummies(df[colname],prefix=colname)
    df=pd.concat([df,col_dummies],axis=1)
    df.drop(colname,axis=1,inplace=True)
    return df
for c_features in ['protocol_type','service','flag','su_attempted']:
    categ_vars[c_features]=categ_vars[c_features].astype('category')
    categ_vars=create_dummies(categ_vars,c_features)
categ_vars.head(3)

In [15]:
network=pd.concat([cont_vars2,categ_vars],axis=1)

In [16]:
network.head(3)

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,num_root,num_file_creations,...,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH,su_attempted_0,su_attempted_1,su_attempted_2
0,0.0,491,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1,0.0,146,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
2,0.0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0


In [17]:
feature_columns=network.columns.difference(['attack'])

In [18]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(network[feature_columns],network['attack'],test_size=0.2,random_state=786)

In [19]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression(verbose=1,n_jobs=-1)

In [20]:
logreg.fit(train_x,train_y)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   35.3s finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=1,
                   warm_start=False)

In [28]:
from sklearn import metrics
print(f'Accuracy-score on training-dataset: {metrics.accuracy_score(train_y,logreg.predict(train_x))}')
print(f'AUC or Concordance-score on training-dataset: {metrics.roc_auc_score(train_y,logreg.predict(train_x))}')
print(f'Accuracy-score on testing-dataset: {metrics.accuracy_score(test_y,logreg.predict(test_x))}')
print(f'AUC or Concordance-score on training-dataset: {metrics.roc_auc_score(test_y,logreg.predict(test_x))}')

Accuracy-score on training-dataset: 0.8959594355911012
AUC or Concordance-score on training-dataset: 0.8920481545973522
Accuracy-score on testing-dataset: 0.8958920420718397
AUC or Concordance-score on training-dataset: 0.8925634983755116


# ---------------------------------------------------------------------------------------------------------

2. Multinomial classification: Activity is normal or DOS or PROBE or R2L or U2R

In [29]:
trn_data2=data_tr.copy()

In [30]:
trn_data2['attack'].replace(['normal'],'0',inplace=True)
trn_data2['attack'].replace(['DoS'],'1',inplace=True)
trn_data2['attack'].replace(['Probe'],'2',inplace=True)
trn_data2['attack'].replace(['R2L'],'3',inplace=True)
trn_data2['attack'].replace(['U2R'],'4',inplace=True)

In [31]:
cont_vars3=trn_data2[['duration','src_bytes','dst_bytes','wrong_fragment','urgent','hot','num_failed_logins','num_compromised',
                   'num_root','num_file_creations','num_shells','num_access_files','num_outbound_cmds','count','srv_count',
                   'serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate',
                   'srv_diff_host_rate','dst_host_count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate',
                   'dst_host_same_src_port_rate','dst_host_srv_diff_host_rate','dst_host_serror_rate',
                   'dst_host_srv_serror_rate','dst_host_rerror_rate','dst_host_srv_rerror_rate','last_flag']]
categ_vars2=trn_data2[['protocol_type','service','flag','attack','land','logged_in','root_shell','su_attempted','is_host_login',
                     'is_guest_login',]]

In [32]:
# Outlier-handling
cont_vars3=cont_vars3.apply(lambda x: x.clip(lower=x.quantile(0.01),upper=x.quantile(0.99)))

In [34]:
def create_dummies(df,colname):
    col_dummies=pd.get_dummies(df[colname],prefix=colname)
    df=pd.concat([df,col_dummies],axis=1)
    df.drop(colname,axis=1,inplace=True)
    return df
for c_features in ['protocol_type','service','flag','su_attempted']:
    categ_vars2[c_features]=categ_vars2[c_features].astype('category')
    categ_vars2=create_dummies(categ_vars2,c_features)
categ_vars2.head(3)

D:\Anaconda3.7_installed\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,attack,land,logged_in,root_shell,is_host_login,is_guest_login,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,service_IRC,...,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH,su_attempted_0,su_attempted_1,su_attempted_2
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,1,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,1,0,0
2,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0


In [35]:
network2=pd.concat([cont_vars3,categ_vars2],axis=1)

In [37]:
network2.head(3)

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,num_root,num_file_creations,...,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH,su_attempted_0,su_attempted_1,su_attempted_2
0,0.0,491,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1,0.0,146,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
2,0.0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0


In [41]:
from sklearn.preprocessing import label_binarize

feature_columns_multi=network2.columns.difference(['attack'])
x1, y1 = network2[feature_columns_multi], network2['attack']

y = label_binarize(y1, classes=[0,1,2,3,4])
n_classes = 5

In [42]:
train_xm,test_xm,train_ym,test_ym=train_test_split(x1,y1,test_size=0.2,random_state=786)

In [44]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

ovr = OneVsRestClassifier(LinearSVC(random_state=0))
ym_score = ovr.fit(train_xm,train_ym).decision_function(test_xm)

D:\Anaconda3.7_installed\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda3.7_installed\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda3.7_installed\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda3.7_installed\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda3.7_installed\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iteration

In [45]:
ym_score

array([[ 1.09486790e+00, -2.66918619e+00, -9.22753349e-01,
        -3.08264377e+00, -5.24890571e+00],
       [-2.21654005e+00,  2.52057370e+00, -6.32860718e+00,
        -6.05797971e+01, -8.57783588e+00],
       [ 2.96888784e+00, -6.37546054e+00, -6.11139508e+00,
        -2.02725466e+00, -7.77853438e+00],
       ...,
       [ 5.56123535e+00, -1.11028763e+02, -3.73442006e+00,
        -2.66174369e+00, -3.73826186e+00],
       [ 1.46782974e+00, -1.26749375e+00, -5.83062605e+00,
        -2.54368622e+01, -6.30712740e+00],
       [-9.19611953e-01,  3.53237223e-03, -9.40251979e-01,
        -1.37191062e+01, -3.03207038e+00]])

In [46]:
ovr.score(train_xm,train_ym)

0.9589196054694477

In [53]:
print(f'Accuracy-score on training-dataset: {metrics.accuracy_score(train_ym,ovr.predict(train_xm))}')
print(f'Accuracy-score on testing-dataset: {metrics.accuracy_score(test_ym,ovr.predict(test_xm))}')

Accuracy-score on training-dataset: 0.9589196054694477
Accuracy-score on testing-dataset: 0.9567771383210955


Since we can't use "roc_auc" or AUC-score as a single summary metric for multiclass models (At most, we may calculate per-class roc_auc). So, we will use confusion-matrix approach to evaluate this multiclass-model. 

In [51]:
pred_ym=ovr.predict(test_xm)

In [52]:
from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(test_ym,pred_ym)
print('Confusion Matrix\n')
print(confusion)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(test_ym,pred_ym)))

from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(test_ym,pred_ym,target_names=['normal','DoS','Probe','R2L','U2R']))

Confusion Matrix

[[12785   476   101    30     0]
 [  148  8947    48   107     0]
 [   74    52  2211     0     0]
 [   37     0     0   163     0]
 [    8     0     1     7     0]]

Accuracy: 0.96


Classification Report

              precision    recall  f1-score   support

      normal       0.98      0.95      0.97     13392
         DoS       0.94      0.97      0.96      9250
       Probe       0.94      0.95      0.94      2337
         R2L       0.53      0.81      0.64       200
         U2R       0.00      0.00      0.00        16

    accuracy                           0.96     25195
   macro avg       0.68      0.74      0.70     25195
weighted avg       0.96      0.96      0.96     25195



D:\Anaconda3.7_installed\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
